In [1]:
import numpy as np
import urllib
import nltk
from nltk.stem.porter import *
from sklearn import linear_model

import string

In [15]:
def parseData(fname):
    for l in urllib.request.urlopen(fname):
        yield eval(l)

all_data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))

data = all_data[:5000]
corpus = [d['review/text'] for d in data]

In [16]:
# text = data[0]['review/text']
# t1 = text2bigrams(data[0]['review/text'])
# t2 = text2bigrams(data[1]['review/text'])

In [17]:
##########################
# 1
##########################

punctuation = set(string.punctuation)

def puncFilter(text):
    return ''.join([c for c in text.lower() if c not in punctuation])

def text2bigrams(text):
    return nltk.bigrams(puncFilter(text).split())

from collections import Counter
bigrams_cnt = Counter()

for text in corpus:
    bigrams_cnt += Counter(text2bigrams(text))
    
print(bigrams_cnt.most_common(5))

[(('with', 'a'), 4587), (('in', 'the'), 2595), (('of', 'the'), 2245), (('is', 'a'), 2056), (('on', 'the'), 2033)]


In [18]:
##########################
# 2
##########################

def feature(text):
    feat = [0] * len(bigrams)
    r = text2bigrams(text)
    for bi in r:
        try:
            feat[bigrams.index(bi)] += 1
        except:
            pass
    feat.append(1)
    return feat

bigrams = [entry[0] for entry in bigrams_cnt.most_common(1000)]
X = [feature(text) for text in corpus]
y = [d['review/overall'] for d in data]

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
predictions = clf.predict(X)
mse = np.mean((y - predictions)**2)
mse

0.3435151332735128

In [34]:
##########################
# 3
##########################

def text2unigram(text):
    return puncFilter(text).split()

def calTF(word, text, cnvt):
    cnt = Counter(cnvt(text))
    try:
        return cnt[word]
    except:
        return 0
    
import math
def calIDF(word, cnvt):
    numer = len(corpus)
    denom = sum([word in cnvt(text) for text in corpus]) 
    return math.log10(numer / denom)
    
def calTF_IDF(word, text, cnvt):
    tf = calTF(word, text, cnvt)
    idf = calIDF(word, cnvt)
    return tf * idf


word_list = ['foam', 'smell', 'banana', 'lactic', 'tart']
text = corpus[0]
for word in word_list:
    tf_idf = calTF_IDF(word, text, text2unigram)
    print('"%s" tf-idf score: %f' % (word, tf_idf))

"foam" tf-idf score: 2.275737
"smell" tf-idf score: 0.537902
"banana" tf-idf score: 3.355561
"lactic" tf-idf score: 5.841638
"tart" tf-idf score: 1.806875


In [36]:
##########################
# 4
##########################

text1 = corpus[0]
text2 = corpus[1]

bigram1 = text2bigrams(text1)
bigram2 = text2bigrams(text2)

tf_idf1 = {w:calTF_IDF(w, text1, text2bigrams) for w in bigram1}
tf_idf2 = {w:calTF_IDF(w, text2, text2bigrams) for w in bigram2}

In [ ]:
##########################
# 5
##########################



In [ ]:
##########################
# 6
##########################

In [40]:
##########################
# 7
##########################

data = all_data
corpus = [d['review/text'] for d in data]

def split_data(X, Y, n_train, n_val, n_test, shuffle=False):
    m = len(X)
    n_val += n_train
    n_test += n_val

    if shuffle:
        r = list(zip(X, Y))
        random.shuffle(r)
        X, Y = list(zip(*r))

    return X[:n_train], Y[:n_train], X[n_train:n_val], \
        Y[n_train:n_val], X[n_val:n_test], Y[n_val:n_test]

X = [feature(text) for text in corpus]
y = [d['review/overall'] for d in data]

trainX, trainY, valX, valY, testX, testY = split_data(X, y, 5000, 5000, 5000, shuffle=True)
# print(len(trainX), len(trainY), len(valX), len(valY), len(testX), len(testY))

5000 5000 6000 6000 7000 7000


In [ ]:
lamda = [0.01, 0.1, 1, 10, 100]
